In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from thefuzz.process import extractOne
from tqdm import tqdm

tqdm.pandas()

In [2]:
stats = pd.read_csv(Path.cwd()/'prepped/player_stats2.csv', index_col=False)
transfers = pd.read_csv(Path.cwd()/'prepped/data.csv', index_col=False).drop(columns=['position'])
elo = pd.read_csv(Path.cwd()/'prepped/elo.csv', index_col=False).drop(columns=['Unnamed: 0'])
display(stats, transfers, elo)

,playerid,player,league,team,season,pos,marketval,matchesplayed,minsplayed,goals,...,prevgoals90,bpassesrcvd90,exits90,gkaerduels90,fk90,directfk90,directfkontrgtpct,corners90,pens,pencnvrspct
0,246963.0,C. Morris,GB1,Luton Town,23/24,CF,13.0,36,2987,10,...,NaN,0.03,NaN,NaN,0.00,0.00,0.0,0.00,4,100.000
1,395516.0,M. Diaby,GB1,Aston Villa,23/24,"CF, AMF",60.0,36,2249,6,...,NaN,0.24,NaN,NaN,0.00,0.00,0.0,0.04,0,0.000
2,525247.0,M. Kilman,GB1,Wolverhampton Wanderers,23/24,"RCB, LCB, CB",35.0,36,3687,2,...,NaN,1.15,NaN,NaN,0.00,0.00,0.0,0.00,0,0.000
3,495666.0,W. Saliba,GB1,Arsenal,23/24,RCB,75.0,36,3656,2,...,NaN,1.35,NaN,NaN,0.00,0.00,0.0,0.00,0,0.000
4,258889.0,D. Solanke,GB1,Bournemouth,23/24,CF,25.0,36,3584,18,...,NaN,0.00,NaN,NaN,0.00,0.00,0.0,0.00,3,66.667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31295,31159.0,M. Harnik,L1,Werder Bremen,18/19,"CF, RWF",2.0,18,888,4,...,NaN,0.30,NaN,NaN,0.00,0.00,0.0,0.00,0,0.000
31296,39094.0,S. Langkamp,L1,Werder Bremen,18/19,"RCB, LCB",2.0,21,1501,0,...,NaN,3.36,NaN,NaN,0.00,0.00,0.0,0.00,0,0.000
31297,19819.0,Raffael,L1,Borussia M'gladbach,18/19,CF,2.5,13,556,2,...,NaN,0.00,NaN,NaN,0.32,0.16,0.0,0.65,0,0.000
31298,31041.0,F. Johnson,L1,Borussia M'gladbach,18/19,"RB, RWF, LW",3.0,18,1068,1,...,NaN,1.10,NaN,NaN,0.00,0.00,0.0,0.00,0,0.000


,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,window,fee,loan
0,Neymar,25,2017,100.00,Spain,LaLiga,FC Barcelona,France,Ligue 1,Paris Saint-Germain,summer,222.00,False
1,Ousmane Dembélé,20,2017,33.00,Germany,Bundesliga,Borussia Dortmund,Spain,LaLiga,FC Barcelona,summer,135.00,False
2,Romelu Lukaku,24,2017,50.00,England,Premier League,Everton FC,England,Premier League,Manchester United,summer,84.70,False
3,Álvaro Morata,24,2017,40.00,Spain,LaLiga,Real Madrid,England,Premier League,Chelsea FC,summer,66.00,False
4,Benjamin Mendy,23,2017,13.00,Monaco,Ligue 1,AS Monaco,England,Premier League,Manchester City,summer,57.50,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27982,Juan Mercado,26,2023,0.60,Bolivia,División Profesional Clausura,Club Deportivo Oriente Petrolero,Bolivia,División Profesional Clausura,Universitario de Vinto,winter,0.00,False
27983,Horacio Torres,24,2023,0.60,Mexico,Liga Expansión MX Cl.,Mineros de Zacatecas,Mexico,Liga Expansión MX Cl.,Atlético La Paz,winter,0.00,False
27984,Joe Gauci,23,2023,0.55,Australia,A-League Men,Adelaide United,England,Premier League,Aston Villa,winter,1.50,False
27985,Shihao Wei,28,2023,0.55,China,Super League,Wuhan Three Towns,China,Super League,Chengdu Rongcheng,winter,0.64,False


,season,league,Club,Elo,avg_elo
0,17/18,GB1,Chelsea,1909,1651.333333
1,17/18,GB1,Tottenham,1887,1651.333333
2,17/18,GB1,Man City,1866,1651.333333
3,17/18,GB1,Man United,1857,1651.333333
4,17/18,GB1,Arsenal,1848,1651.333333
...,...,...,...,...,...
2054,23/24,SC1,Kilmarnock,1236,1319.923077
2055,23/24,SC1,St Johnstone,1235,1319.923077
2056,23/24,SC1,Ross County,1221,1319.923077
2057,23/24,SC1,Dundee United,1198,1319.923077


In [3]:
league_map = {
    'LaLiga': 'ES1',
    'Bundesliga': 'L1',
    'Premier League': 'GB1',
    'Ligue 1': 'FR1',
    'Serie A': 'IT1',
    'Eredivisie': 'NL1',
    'Liga Portugal': 'PO1',
    'Liga NOS': 'PO1',
    'Jupiler Pro League': 'BE1',
    'JPL': 'BE1',
    'Superliga': 'DK1',
    'Super League': 'GR1',
    'Premier Liga': 'RU1',
    'Premiership': 'SC1',
    'Süper Lig': 'TR1'
}

In [4]:
transfers.league_from = transfers.league_from.replace(league_map)
transfers.league_to = transfers.league_to.replace(league_map)
transfers.country_from = transfers.country_from.replace('Monaco', 'France')
transfers.country_to = transfers.country_to.replace('Monaco', 'France')

In [5]:
transfers.iloc[14]

name            Corentin Tolisso
age                           22
season                      2017
market_value                22.0
country_from              France
league_from                  FR1
club_from         Olympique Lyon
country_to               Germany
league_to                     L1
club_to            Bayern Munich
window                    summer
fee                         41.5
loan                       False
Name: 14, dtype: object

In [6]:
def short_name_gen(name):
    name = name.split(' ')
    if len(name) == 1: 
        return name[0]
    name[0] = name[0][0] + '.'
    return ' '.join(name)

def match_wyscout_to_tm(row: pd.Series, thresh_player=50, thres_team=0): 
    short_name = short_name_gen(row['name'])
    
    club_from_wyscout = extractOne(
        query=row.club_from, score_cutoff=thres_team,
        choices=stats[stats['league'] == row.league_from]['team'].unique().tolist())[0]
    
    club_to_wyscout = extractOne(
        query=row.club_to, score_cutoff=thres_team,
        choices=stats[stats['league'] == row.league_to]['team'].unique().tolist())[0]
    
    name_wyscout = extractOne(
        query=row['name'], score_cutoff=thresh_player,
        choices=stats[(stats['league'] == row.league_to) & (stats['team'] == club_to_wyscout) & (stats['season'] == row.season)]['player'].unique().tolist()
        ) or (None, 0)
    
    short_name_wyscout = extractOne(
        query=short_name, score_cutoff=thresh_player,
        choices=stats[(stats[f'league'] == row.league_to) & (stats['team'] == club_to_wyscout)& (stats['season'] == row.season)]['player'].unique().tolist()
        ) or (None, 0)
    
    if club_from_wyscout == None or club_to_wyscout == None or (name_wyscout[0] == None and short_name_wyscout[0] == None):
        row['name'] = None
        row['playerid'] = None
        return row
    else:
        row['name'] = max([name_wyscout, short_name_wyscout], key = lambda t: t[1])[0]
        if stats[(stats[f'league'] == row.league_to) & (stats['team'] == club_to_wyscout) & (stats['season'] == row.season) & (stats['player'] == row['name'])]['playerid'].size > 1:
            display(row.club_to, club_to_wyscout, row['name'], short_name, name_wyscout, short_name_wyscout)
            display(stats[(stats[f'league'] == row.league_to) & (stats['team'] == club_to_wyscout) & (stats['player'] == row['name'])])
            row['playerid'] = np.nan
            return row
        row['playerid'] = stats[(stats[f'league'] == row.league_to) & (stats['team'] == club_to_wyscout) & (stats['season'] == row.season) & (stats['player'] == row['name'])]['playerid'].item()
        row['club_from'] = club_from_wyscout
        row['club_to'] = club_to_wyscout

    return row

def match_elo_to_tm(row, thresh_team=0):
    club_from_elo = extractOne(
        query=row.club_from,
        choices=elo[(elo.league == row.league_from) & (elo.season == row.season)]['Club'].unique().tolist())[0]
    # print(club_from_elo)
    club_to_elo = extractOne(
        query=row.club_to,
        choices=elo[(elo.league == row.league_to) & (elo.season == row.season)]['Club'].unique().tolist())[0]
    # print(club__elo)
    row['club_from_elo'] = elo[(elo.league == row.league_from) & (elo.season == row.season) & (elo.Club == club_from_elo)]['Elo'].item()
    row['club_to_elo'] = elo[(elo.league == row.league_to) & (elo.season == row.season) & (elo.Club == club_to_elo)]['Elo'].item()
    row['league_from_elo'] = elo[(elo.league == row.league_from) & (elo.season == row.season) & (elo.Club == club_from_elo)]['avg_elo'].item()
    row['league_to_elo'] = elo[(elo.league == row.league_to) & (elo.season == row.season) & (elo.Club == club_to_elo)]['avg_elo'].item()
    return row

In [7]:
competitions = ['GB1', 'FR1', 'TR1', 'NL1', 'SC1', 'IT1', 'DK1', 'PO1', 'ES1', 'BE1', 'RU1', 'GR1', 'L1']
countries = ['Spain', 'Germany', 'England', 'France', 'Italy', 'Netherlands', 'Portugal', 'Belgium', 
             'Denmark', 'Greece', 'Russia', 'Scotland', 'Türkiye']
transfers = transfers[transfers.league_to.isin(competitions) &
                      transfers.country_to.isin(countries)]
transfers = transfers[transfers.league_from.isin(competitions) & 
                      transfers.country_from.isin(countries)]
transfers['season'] = transfers['season'].map({2018: '18/19', 2019: '19/20', 2020: '20/21', 2021: '21/22', 2022: '22/23', 2023: '23/24'})
transfers = transfers[transfers.season.notnull()] # remove 17/18 season
# transfers['club_to'] = transfers.club_to.str.replace('Inter Milan', 'Internazionale') \
#                                         .str.replace('Paris Saint-Germain', 'PSG') \
#                                         .str.replace('Espanyol Barcelona', 'Espanyol')
# transfers['club_from'] = transfers.club_from.str.replace('Inter Milan', 'Internazionale') \
#                                         .str.replace('Paris Saint-Germain', 'PSG') \
#                                         .str.replace('Espanyol Barcelona', 'Espanyol')

In [8]:
transfers

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,window,fee,loan
4000,Kylian Mbappé,19,18/19,120.00,France,FR1,AS Monaco,France,FR1,Paris Saint-Germain,summer,180.0,False
4001,Cristiano Ronaldo,33,18/19,100.00,Spain,ES1,Real Madrid,Italy,IT1,Juventus FC,summer,117.0,False
4002,Kepa Arrizabalaga,23,18/19,20.00,Spain,ES1,Athletic Bilbao,England,GB1,Chelsea FC,summer,80.0,False
4003,Thomas Lemar,22,18/19,65.00,France,FR1,AS Monaco,Spain,ES1,Atlético de Madrid,summer,72.0,False
4004,Riyad Mahrez,27,18/19,50.00,England,GB1,Leicester City,England,GB1,Manchester City,summer,67.8,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27695,Sylla Sow,27,23/24,0.65,Netherlands,NL1,Go Ahead Eagles,Netherlands,NL1,NEC Nijmegen,winter,0.0,False
27811,Ertaç Özbir,34,23/24,0.60,Türkiye,TR1,Adana Demirspor,Türkiye,TR1,MKE Ankaragücü,winter,0.0,False
27829,Igor Yurganov,30,23/24,0.60,Russia,RU1,FC Sochi,Russia,RU1,Fakel Voronezh,winter,0.0,False
27832,Fredrik Oppegård,21,23/24,0.60,Netherlands,NL1,PSV Eindhoven,Netherlands,NL1,Heracles Almelo,winter,0.0,True


In [9]:
def parse_names(x, full=False):
        splitter = x.find('.')
        if splitter == -1:
            return x[:len(x)//2]
        elif(x.count('.') > 1):
            x = x.split('.')
            if full:
                return '.'.join(x[len(x)//2:])[:-1]
            else:
                return  '.'.join(x[:len(x)//2])[:-1]
        else:
            if full: 
                return x[splitter-1:]
            else:
                return x[:splitter-1]
            
def link_name(market, stats):
    x = market['short_name'].apply(lambda x: extractOne(x, stats.Player.unique().tolist()))
    y = market['name'].apply(lambda x: extractOne(x, stats.Player.unique().tolist()))
    link = []
    for i in range(len(x)):
        if x.iloc[i][1] > y.iloc[i][1]:
            name = x.iloc[i][0]
            score = x.iloc[i][1]
        else:
            name = y.iloc[i][0]
            score = y.iloc[i][1]

        if score < 90:
            link.append((np.nan, score))
        else:
            link.append((name, score))
        
    return pd.Series(link).apply(lambda x: x[0])

def get_season(x):
    year_to_season = {
        2018 : '18/19',
        2019 : '19/20',
        2020 : '20/21',
        2021 : '21/22',
        2022 : '22/23',
        2023 : '23/24',
    }
    date_ser = pd.to_datetime(x)
    year = date_ser.year if (date_ser.month >= 8 and date_ser.day >= 1) else date_ser.year - 1
    return year_to_season[year]

id_to_league = {
    'GB1' : 'premier-league',
    'FR1' : 'ligue-1',
    'L1' : 'bundesliga',
    'ES1' : 'laliga',
    'IT1' :'serie-a',
}

league_to_league_name = {
    'premier-league': 'Premier League',
    'ligue-1': 'Ligue 1',
    'bundesliga': 'Bundesliga',
    'laliga': 'LaLiga',
    'serie-a': 'Serie A',
}

In [10]:
apps = pd.read_csv(Path.cwd()/'archive-4/appearances.csv')
apps = apps[(apps.competition_id.isin(['GB1', 'FR1', 'TR1', 'NL1', 'SC1', 'IT1', 'DK1', 'PO1', 'ES1', 'BE1', 'RU1', 'GR1', 'L1'])) & (apps.date > '2018-08-01')]
apps['season'] = apps['date'].apply(get_season)

apps = apps.groupby(['player_id', 'player_club_id', 'season', 'competition_id']).first().reset_index()[['player_id', 'player_club_id', 'season', 'competition_id']]
market = pd.read_csv(Path.cwd()/'archive-4/player_valuations.csv')
players = pd.read_csv(Path.cwd()/'archive-4/players.csv')
comps = pd.read_csv(Path.cwd()/'archive-4/competitions.csv')
teams = pd.read_csv(Path.cwd()/'archive-4/clubs.csv')

market = market[market.date >'2018-08-01']
market['season'] = market['date'].apply(get_season)
market = market.merge(apps, how='left', on=['player_id', 'season'])
market = market.merge(teams[['club_id', 'club_code', 'name']].rename({'name': 'club_name'}, axis=1), how='left', left_on='player_club_id', right_on='club_id')
market = market.groupby(by=['player_id', 'player_club_id', 'season', 'competition_id']).last().reset_index()


players['short_name'] = players['first_name'].str[0].fillna('') + '. ' + players['last_name']


market = market.merge(players[['short_name', 'player_id', 'name']], how='inner', on='player_id').dropna()
market['league'] = market['competition_id']
market = market[['player_id', 'season', 'club_code', 'club_name', 'short_name', 'name', 'league', 'market_value_in_eur']]

In [11]:
market[market.name == 'William Carvalho']

,player_id,season,club_code,club_name,short_name,name,league,market_value_in_eur
7327,100131,18/19,real-betis-sevilla,Real Betis Balompié S.A.D.,W. Carvalho,William Carvalho,ES1,25000000
7328,100131,19/20,real-betis-sevilla,Real Betis Balompié S.A.D.,W. Carvalho,William Carvalho,ES1,16000000
7329,100131,20/21,real-betis-sevilla,Real Betis Balompié S.A.D.,W. Carvalho,William Carvalho,ES1,15000000
7330,100131,21/22,real-betis-sevilla,Real Betis Balompié S.A.D.,W. Carvalho,William Carvalho,ES1,16000000
7331,100131,22/23,real-betis-sevilla,Real Betis Balompié S.A.D.,W. Carvalho,William Carvalho,ES1,14000000
7332,100131,23/24,real-betis-sevilla,Real Betis Balompié S.A.D.,W. Carvalho,William Carvalho,ES1,6000000


In [12]:
def player_id(row, thresh=90):
    m = market[(market.season == row['season']) & (market.league == row['league_to'])]
    club = extractOne(row.club_to, choices=m.club_code.unique())[0]
    m = m[m.club_code == club]
    tm_short = extractOne(query=row['name'], choices=m.short_name, score_cutoff=thresh) or (None, 0)
    tm_long = extractOne(query=row['name'], choices=m['name'], score_cutoff=thresh) or (None, 0)
        
    if tm_short == (None, 0) and tm_long == (None, 0):
        m = market[(market.season == row['season']) & (market.league == row['league_to'])]
        club = extractOne(row.club_to, choices=m.club_name.unique())[0]
        m = m[m.club_name == club]
        tm_short = extractOne(query=row['name'], choices=m.short_name, score_cutoff=thresh) or (None, 0)
        tm_long = extractOne(query=row['name'], choices=m['name'], score_cutoff=thresh) or (None, 0)
        if tm_short == (None, 0) and tm_long == (None, 0):
            row['playerid'] = np.nan
            return row
    
    if m[m.short_name == tm_short[0]].shape[0] > 1 or m[m['name'] == tm_long[0]].shape[0] > 1:
        display(row, m[m.short_name == tm_short[0]], m[m['name'] == tm_long[0]])
    if tm_short[1] > tm_long[1]:
        row['playerid'] = m[m.short_name == tm_short[0]]['player_id'].item()
    else:
        row['playerid'] = m[m['name'] == tm_long[0]]['player_id'].item()

    return row

In [13]:
transfers = transfers.progress_apply(player_id, axis=1)

  0%|          | 3/5829 [00:00<03:21, 28.91it/s]

100%|██████████| 5829/5829 [01:40<00:00, 57.99it/s]


In [14]:
transfers = transfers.dropna(subset=['playerid'])

In [15]:
transfers = transfers.progress_apply(match_elo_to_tm, axis=1)

100%|██████████| 5296/5296 [01:23<00:00, 63.68it/s]


In [16]:
transfers

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,window,fee,loan,playerid,club_from_elo,club_to_elo,league_from_elo,league_to_elo
4000,Kylian Mbappé,19,18/19,120.0,France,FR1,AS Monaco,France,FR1,Paris Saint-Germain,summer,180.0,False,342229.0,1771,1881,1561.200000,1561.200000
4001,Cristiano Ronaldo,33,18/19,100.0,Spain,ES1,Real Madrid,Italy,IT1,Juventus FC,summer,117.0,False,8198.0,2023,1961,1685.533333,1591.500000
4002,Kepa Arrizabalaga,23,18/19,20.0,Spain,ES1,Athletic Bilbao,England,GB1,Chelsea FC,summer,80.0,False,192279.0,1700,1836,1685.533333,1665.233333
4003,Thomas Lemar,22,18/19,65.0,France,FR1,AS Monaco,Spain,ES1,Atlético de Madrid,summer,72.0,False,205562.0,1771,1931,1561.200000,1685.533333
4004,Riyad Mahrez,27,18/19,50.0,England,GB1,Leicester City,England,GB1,Manchester City,summer,67.8,False,171424.0,1696,1974,1665.233333,1665.233333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27048,Yvandro Borges Sanches,19,23/24,1.0,Germany,L1,Borussia Mönchengladbach,Netherlands,NL1,NEC Nijmegen,winter,0.0,True,586659.0,1684,1441,1618.066667,1490.285714
27055,Jason van Duiven,18,23/24,1.0,Netherlands,NL1,PSV Eindhoven,Netherlands,NL1,Almere City FC,winter,0.0,True,738478.0,1745,1405,1490.285714,1490.285714
27107,Umut Nayir,30,23/24,1.0,Türkiye,TR1,Fenerbahce,Türkiye,TR1,Pendikspor,winter,0.0,True,248271.0,1639,1316,1435.363636,1435.363636
27169,Mamadou Sarr,18,23/24,1.0,France,FR1,Olympique Lyon,Belgium,BE1,RWD Molenbeek,winter,0.0,True,910905.0,1696,1341,1552.633333,1474.736842


In [17]:
# transfers = transfers.progress_apply(match_wyscout_to_tm, axis=1).dropna(subset=['name'])
# transfers

In [18]:
prev_season = {'18/19': '17/18', '19/20': '18/19', '20/21': '19/20', '21/22': '20/21', '22/23': '21/22', '23/24': '22/23'}

In [19]:
stats

,playerid,player,league,team,season,pos,marketval,matchesplayed,minsplayed,goals,...,prevgoals90,bpassesrcvd90,exits90,gkaerduels90,fk90,directfk90,directfkontrgtpct,corners90,pens,pencnvrspct
0,246963.0,C. Morris,GB1,Luton Town,23/24,CF,13.0,36,2987,10,...,NaN,0.03,NaN,NaN,0.00,0.00,0.0,0.00,4,100.000
1,395516.0,M. Diaby,GB1,Aston Villa,23/24,"CF, AMF",60.0,36,2249,6,...,NaN,0.24,NaN,NaN,0.00,0.00,0.0,0.04,0,0.000
2,525247.0,M. Kilman,GB1,Wolverhampton Wanderers,23/24,"RCB, LCB, CB",35.0,36,3687,2,...,NaN,1.15,NaN,NaN,0.00,0.00,0.0,0.00,0,0.000
3,495666.0,W. Saliba,GB1,Arsenal,23/24,RCB,75.0,36,3656,2,...,NaN,1.35,NaN,NaN,0.00,0.00,0.0,0.00,0,0.000
4,258889.0,D. Solanke,GB1,Bournemouth,23/24,CF,25.0,36,3584,18,...,NaN,0.00,NaN,NaN,0.00,0.00,0.0,0.00,3,66.667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31295,31159.0,M. Harnik,L1,Werder Bremen,18/19,"CF, RWF",2.0,18,888,4,...,NaN,0.30,NaN,NaN,0.00,0.00,0.0,0.00,0,0.000
31296,39094.0,S. Langkamp,L1,Werder Bremen,18/19,"RCB, LCB",2.0,21,1501,0,...,NaN,3.36,NaN,NaN,0.00,0.00,0.0,0.00,0,0.000
31297,19819.0,Raffael,L1,Borussia M'gladbach,18/19,CF,2.5,13,556,2,...,NaN,0.00,NaN,NaN,0.32,0.16,0.0,0.65,0,0.000
31298,31041.0,F. Johnson,L1,Borussia M'gladbach,18/19,"RB, RWF, LW",3.0,18,1068,1,...,NaN,1.10,NaN,NaN,0.00,0.00,0.0,0.00,0,0.000


In [20]:
transfers['prev_season'] = transfers.season.map(prev_season)

In [21]:
transfers

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,window,fee,loan,playerid,club_from_elo,club_to_elo,league_from_elo,league_to_elo,prev_season
4000,Kylian Mbappé,19,18/19,120.0,France,FR1,AS Monaco,France,FR1,Paris Saint-Germain,summer,180.0,False,342229.0,1771,1881,1561.200000,1561.200000,17/18
4001,Cristiano Ronaldo,33,18/19,100.0,Spain,ES1,Real Madrid,Italy,IT1,Juventus FC,summer,117.0,False,8198.0,2023,1961,1685.533333,1591.500000,17/18
4002,Kepa Arrizabalaga,23,18/19,20.0,Spain,ES1,Athletic Bilbao,England,GB1,Chelsea FC,summer,80.0,False,192279.0,1700,1836,1685.533333,1665.233333,17/18
4003,Thomas Lemar,22,18/19,65.0,France,FR1,AS Monaco,Spain,ES1,Atlético de Madrid,summer,72.0,False,205562.0,1771,1931,1561.200000,1685.533333,17/18
4004,Riyad Mahrez,27,18/19,50.0,England,GB1,Leicester City,England,GB1,Manchester City,summer,67.8,False,171424.0,1696,1974,1665.233333,1665.233333,17/18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27048,Yvandro Borges Sanches,19,23/24,1.0,Germany,L1,Borussia Mönchengladbach,Netherlands,NL1,NEC Nijmegen,winter,0.0,True,586659.0,1684,1441,1618.066667,1490.285714,22/23
27055,Jason van Duiven,18,23/24,1.0,Netherlands,NL1,PSV Eindhoven,Netherlands,NL1,Almere City FC,winter,0.0,True,738478.0,1745,1405,1490.285714,1490.285714,22/23
27107,Umut Nayir,30,23/24,1.0,Türkiye,TR1,Fenerbahce,Türkiye,TR1,Pendikspor,winter,0.0,True,248271.0,1639,1316,1435.363636,1435.363636,22/23
27169,Mamadou Sarr,18,23/24,1.0,France,FR1,Olympique Lyon,Belgium,BE1,RWD Molenbeek,winter,0.0,True,910905.0,1696,1341,1552.633333,1474.736842,22/23


In [22]:
t = transfers.merge(stats, left_on=['prev_season', 'playerid'], right_on=['season', 'playerid'], suffixes=('', '_x')).drop(columns=['prev_season', 'season_x'])

In [23]:
t

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,...,prevgoals90,bpassesrcvd90,exits90,gkaerduels90,fk90,directfk90,directfkontrgtpct,corners90,pens,pencnvrspct
0,Antoine Griezmann,28,19/20,130.0,Spain,ES1,Atlético de Madrid,Spain,ES1,FC Barcelona,...,NaN,0.03,NaN,NaN,1.20,0.40,40.00,1.63,3,100.0
1,João Félix,19,19/20,70.0,Portugal,PO1,SL Benfica,Spain,ES1,Atlético de Madrid,...,NaN,0.05,NaN,NaN,0.05,0.00,0.00,0.05,0,0.0
2,Philippe Coutinho,27,19/20,90.0,Spain,ES1,FC Barcelona,Germany,L1,Bayern Munich,...,NaN,0.13,NaN,NaN,0.34,0.13,33.33,2.81,0,0.0
3,Harry Maguire,26,19/20,50.0,England,GB1,Leicester City,England,GB1,Manchester United,...,NaN,1.00,NaN,NaN,0.00,0.00,0.00,0.00,0,0.0
4,Frenkie de Jong,22,19/20,85.0,Netherlands,NL1,Ajax Amsterdam,Spain,ES1,FC Barcelona,...,NaN,0.72,NaN,NaN,0.03,0.00,0.00,0.10,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3738,Kenzo Goudmijn,22,23/24,1.3,Netherlands,NL1,AZ Alkmaar,Netherlands,NL1,Excelsior Rotterdam,...,NaN,0.46,NaN,NaN,0.03,0.03,0.00,0.00,0,0.0
3739,Daniel Schmidt,31,23/24,1.2,Belgium,BE1,Sint-Truidense VV,Belgium,BE1,KAA Gent,...,-0.049,0.00,2.11,0.72,NaN,NaN,NaN,0.00,0,0.0
3740,Mats Seuntjens,31,23/24,1.0,Netherlands,NL1,FC Utrecht,Netherlands,NL1,RKC Waalwijk,...,NaN,0.13,NaN,NaN,0.56,0.10,66.67,1.86,0,0.0
3741,Yvandro Borges Sanches,19,23/24,1.0,Germany,L1,Borussia Mönchengladbach,Netherlands,NL1,NEC Nijmegen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0


In [24]:
t.drop_duplicates()

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,...,prevgoals90,bpassesrcvd90,exits90,gkaerduels90,fk90,directfk90,directfkontrgtpct,corners90,pens,pencnvrspct
0,Antoine Griezmann,28,19/20,130.0,Spain,ES1,Atlético de Madrid,Spain,ES1,FC Barcelona,...,NaN,0.03,NaN,NaN,1.20,0.40,40.00,1.63,3,100.0
1,João Félix,19,19/20,70.0,Portugal,PO1,SL Benfica,Spain,ES1,Atlético de Madrid,...,NaN,0.05,NaN,NaN,0.05,0.00,0.00,0.05,0,0.0
2,Philippe Coutinho,27,19/20,90.0,Spain,ES1,FC Barcelona,Germany,L1,Bayern Munich,...,NaN,0.13,NaN,NaN,0.34,0.13,33.33,2.81,0,0.0
3,Harry Maguire,26,19/20,50.0,England,GB1,Leicester City,England,GB1,Manchester United,...,NaN,1.00,NaN,NaN,0.00,0.00,0.00,0.00,0,0.0
4,Frenkie de Jong,22,19/20,85.0,Netherlands,NL1,Ajax Amsterdam,Spain,ES1,FC Barcelona,...,NaN,0.72,NaN,NaN,0.03,0.00,0.00,0.10,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3738,Kenzo Goudmijn,22,23/24,1.3,Netherlands,NL1,AZ Alkmaar,Netherlands,NL1,Excelsior Rotterdam,...,NaN,0.46,NaN,NaN,0.03,0.03,0.00,0.00,0,0.0
3739,Daniel Schmidt,31,23/24,1.2,Belgium,BE1,Sint-Truidense VV,Belgium,BE1,KAA Gent,...,-0.049,0.00,2.11,0.72,NaN,NaN,NaN,0.00,0,0.0
3740,Mats Seuntjens,31,23/24,1.0,Netherlands,NL1,FC Utrecht,Netherlands,NL1,RKC Waalwijk,...,NaN,0.13,NaN,NaN,0.56,0.10,66.67,1.86,0,0.0
3741,Yvandro Borges Sanches,19,23/24,1.0,Germany,L1,Borussia Mönchengladbach,Netherlands,NL1,NEC Nijmegen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0


In [25]:
t2 = t.merge(stats[['playerid', 'season', 'marketval']], left_on=['season', 'playerid'], right_on=['season', 'playerid'], suffixes=('', '_0')).drop_duplicates()

In [26]:
t2 = t2.drop(columns=['player', 'league', 'team'])

In [27]:
t2 = t2[~t2.loan]

In [28]:
t2

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,...,bpassesrcvd90,exits90,gkaerduels90,fk90,directfk90,directfkontrgtpct,corners90,pens,pencnvrspct,marketval_0
0,Antoine Griezmann,28,19/20,130.0,Spain,ES1,Atlético de Madrid,Spain,ES1,FC Barcelona,...,0.03,NaN,NaN,1.20,0.4,40.00,1.63,3,100.0,96.0
1,João Félix,19,19/20,70.0,Portugal,PO1,SL Benfica,Spain,ES1,Atlético de Madrid,...,0.05,NaN,NaN,0.05,0.0,0.00,0.05,0,0.0,81.0
3,Harry Maguire,26,19/20,50.0,England,GB1,Leicester City,England,GB1,Manchester United,...,1.00,NaN,NaN,0.00,0.0,0.00,0.00,0,0.0,56.0
4,Frenkie de Jong,22,19/20,85.0,Netherlands,NL1,Ajax Amsterdam,Spain,ES1,FC Barcelona,...,0.72,NaN,NaN,0.03,0.0,0.00,0.10,0,0.0,72.0
5,Matthijs de Ligt,19,19/20,75.0,Netherlands,NL1,Ajax Amsterdam,Italy,IT1,Juventus FC,...,2.91,NaN,NaN,0.00,0.0,0.00,0.00,0,0.0,67.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4172,Mame Thiam,31,23/24,2.8,Türkiye,TR1,Kayserispor,Türkiye,TR1,Pendikspor,...,0.00,NaN,NaN,0.00,0.0,0.00,0.04,2,50.0,2.8
4173,Yann Gboho,22,23/24,2.5,Belgium,BE1,Cercle Brugge,France,FR1,FC Toulouse,...,0.07,NaN,NaN,0.00,0.0,0.00,0.00,0,0.0,2.5
4175,Kevin Vogt,32,23/24,2.5,Germany,L1,TSG 1899 Hoffenheim,Germany,L1,1.FC Union Berlin,...,2.26,NaN,NaN,0.00,0.0,0.00,0.00,0,0.0,2.5
4183,Daniel Schmidt,31,23/24,1.2,Belgium,BE1,Sint-Truidense VV,Belgium,BE1,KAA Gent,...,0.00,2.11,0.72,NaN,NaN,NaN,0.00,0,0.0,1.2


In [29]:
t2.to_csv('prepped/final-2.csv', index=False)

In [31]:
t2

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,...,bpassesrcvd90,exits90,gkaerduels90,fk90,directfk90,directfkontrgtpct,corners90,pens,pencnvrspct,marketval_0
0,Antoine Griezmann,28,19/20,130.0,Spain,ES1,Atlético de Madrid,Spain,ES1,FC Barcelona,...,0.03,NaN,NaN,1.20,0.4,40.00,1.63,3,100.0,96.0
1,João Félix,19,19/20,70.0,Portugal,PO1,SL Benfica,Spain,ES1,Atlético de Madrid,...,0.05,NaN,NaN,0.05,0.0,0.00,0.05,0,0.0,81.0
3,Harry Maguire,26,19/20,50.0,England,GB1,Leicester City,England,GB1,Manchester United,...,1.00,NaN,NaN,0.00,0.0,0.00,0.00,0,0.0,56.0
4,Frenkie de Jong,22,19/20,85.0,Netherlands,NL1,Ajax Amsterdam,Spain,ES1,FC Barcelona,...,0.72,NaN,NaN,0.03,0.0,0.00,0.10,0,0.0,72.0
5,Matthijs de Ligt,19,19/20,75.0,Netherlands,NL1,Ajax Amsterdam,Italy,IT1,Juventus FC,...,2.91,NaN,NaN,0.00,0.0,0.00,0.00,0,0.0,67.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4172,Mame Thiam,31,23/24,2.8,Türkiye,TR1,Kayserispor,Türkiye,TR1,Pendikspor,...,0.00,NaN,NaN,0.00,0.0,0.00,0.04,2,50.0,2.8
4173,Yann Gboho,22,23/24,2.5,Belgium,BE1,Cercle Brugge,France,FR1,FC Toulouse,...,0.07,NaN,NaN,0.00,0.0,0.00,0.00,0,0.0,2.5
4175,Kevin Vogt,32,23/24,2.5,Germany,L1,TSG 1899 Hoffenheim,Germany,L1,1.FC Union Berlin,...,2.26,NaN,NaN,0.00,0.0,0.00,0.00,0,0.0,2.5
4183,Daniel Schmidt,31,23/24,1.2,Belgium,BE1,Sint-Truidense VV,Belgium,BE1,KAA Gent,...,0.00,2.11,0.72,NaN,NaN,NaN,0.00,0,0.0,1.2


In [ ]:
t[t.aerduels90.isna() & (t.season != '18/19')].to_csv('errors.csv')